# Fine-Tuning Language Models for Text Classification
用于文本分类的微调语言模型

## Preparation for Google Colab

In [ ]:
import os
from google.colab import drive

#  挂载 google 云盘
drive.mount("/content/drive")

print(os.getcwd())  # /content

# print(os.listdir("/content/drive/MyDrive/"))

# print(os.listdir("/content/drive/MyDrive/Colab Notebooks"))

# if os.getcwd() != "/content/drive/MyDrive":
#     os.chdir("/content/drive/MyDrive")

# print(os.getcwd())

In [ ]:
# 提前将 requirements.txt 放在 google 云盘上
!pip install -r /content/drive/MyDrive/requirements.txt

In [3]:
subdir = "ch05a"
work_path = "/content/drive/MyDrive/" + subdir
if not os.path.exists(work_path):
    os.mkdir(work_path)
os.chdir(work_path)
print(os.getcwd())

/content/drive/MyDrive/ch05a


In [4]:
!apt-get install tree && tree -a "./"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 1s (51.7 kB/s)
Selecting previously unselected package tree.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
./

0 directories, 0 files


## Loading pre-trained model and tokenizer

In [5]:
from transformers import DistilBertTokenizerFast

model_path = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [6]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained(
    model_path, id2label={0: "NEG", 1: "POS"}, label2id={"NEG": 0, "POS": 1}
)
model

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


## Loading popular IMDB dataset

In [7]:
from datasets import load_dataset

# to take entire dataset from original train 25 K AND TEST 25K
"""
imdb_train= load_dataset('imdb', split="train")
imdb_test= load_dataset('imdb', split="test[:6250]+test[-6250:]")
imdb_val= load_dataset('imdb', split="test[6250:12500]+test[-12500:-6250]")

"""
# to take smaller portion 4K for train, 1K for test and 1K for validation
imdb_train = load_dataset("imdb", split="train[:2000]+train[-2000:]")
imdb_test = load_dataset("imdb", split="test[:500]+test[-500:]")
imdb_val = load_dataset("imdb", split="test[500:1000]+test[-1000:-500]")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
imdb_train.shape, imdb_test.shape, imdb_val.shape

((4000, 2), (1000, 2), (1000, 2))

In [9]:
imdb_train[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [10]:
imdb_test[1]

{'text': "Worth the entertainment value of a rental, especially if you like action movies. This one features the usual car chases, fights with the great Van Damme kick style, shooting battles with the 40 shell load shotgun, and even terrorist style bombs. All of this is entertaining and competently handled but there is nothing that really blows you away if you've seen your share before.<br /><br />The plot is made interesting by the inclusion of a rabbit, which is clever but hardly profound. Many of the characters are heavily stereotyped -- the angry veterans, the terrified illegal aliens, the crooked cops, the indifferent feds, the bitchy tough lady station head, the crooked politician, the fat federale who looks like he was typecast as the Mexican in a Hollywood movie from the 1940s. All passably acted but again nothing special.<br /><br />I thought the main villains were pretty well done and fairly well acted. By the end of the movie you certainly knew who the good guys were and wer

In [11]:
imdb_val[2]

{'text': "i have one word: focus.<br /><br />well.<br /><br />IMDb wants me to use at least ten lines of text. okay. let's discuss the fine points of focus. i don't know about the rest of you, but in my first year of film school they taught us a lot of useless crap, like 'you'll all be famous avant-garde filmmakers someday'--but they also taught us how to do this crazy thing called FOCUSING the lens! it was amazing! you give a little twist and wham! everything is clear as a bell. the person who shot what alice found needs to learn a few things about the finer points of focus. lighting, too. this movie is not only completely out of focus, it's also lit like the corner of someone's basement.<br /><br />don't even get me started on pacing or plot. they could have trimmed about ten seconds off the beginning and end of every single shot.<br /><br />but who cares about that anyhow? there is not enough lurid in this movie to make up for the utter lack of regard to film's best friends--FOCUS, 

In [12]:
enc_train = imdb_train.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)
enc_test = imdb_test.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)
enc_val = imdb_val.map(
    lambda e: tokenizer(e["text"], padding=True, truncation=True),
    batched=True,
    batch_size=1000,
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
enc_train.shape, enc_test.shape, enc_val.shape

((4000, 4), (1000, 4), (1000, 4))

In [ ]:
enc_train[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [13]:
enc_test[1]

{'text': "Worth the entertainment value of a rental, especially if you like action movies. This one features the usual car chases, fights with the great Van Damme kick style, shooting battles with the 40 shell load shotgun, and even terrorist style bombs. All of this is entertaining and competently handled but there is nothing that really blows you away if you've seen your share before.<br /><br />The plot is made interesting by the inclusion of a rabbit, which is clever but hardly profound. Many of the characters are heavily stereotyped -- the angry veterans, the terrified illegal aliens, the crooked cops, the indifferent feds, the bitchy tough lady station head, the crooked politician, the fat federale who looks like he was typecast as the Mexican in a Hollywood movie from the 1940s. All passably acted but again nothing special.<br /><br />I thought the main villains were pretty well done and fairly well acted. By the end of the movie you certainly knew who the good guys were and wer

In [14]:
enc_val[2]

{'text': "i have one word: focus.<br /><br />well.<br /><br />IMDb wants me to use at least ten lines of text. okay. let's discuss the fine points of focus. i don't know about the rest of you, but in my first year of film school they taught us a lot of useless crap, like 'you'll all be famous avant-garde filmmakers someday'--but they also taught us how to do this crazy thing called FOCUSING the lens! it was amazing! you give a little twist and wham! everything is clear as a bell. the person who shot what alice found needs to learn a few things about the finer points of focus. lighting, too. this movie is not only completely out of focus, it's also lit like the corner of someone's basement.<br /><br />don't even get me started on pacing or plot. they could have trimmed about ten seconds off the beginning and end of every single shot.<br /><br />but who cares about that anyhow? there is not enough lurid in this movie to make up for the utter lack of regard to film's best friends--FOCUS, 

In [15]:
import pandas as pd

pd.DataFrame(enc_train)

,text,label,input_ids,attention_mask
0,I rented I AM CURIOUS-YELLOW from my video sto...,0,"[101, 1045, 12524, 1045, 2572, 8025, 1011, 375...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"""I Am Curious: Yellow"" is a risible and preten...",0,"[101, 1000, 1045, 2572, 8025, 1024, 3756, 1000...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,If only to avoid making this type of film in t...,0,"[101, 2065, 2069, 2000, 4468, 2437, 2023, 2828...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,This film was probably inspired by Godard's Ma...,0,"[101, 2023, 2143, 2001, 2763, 4427, 2011, 2643...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"Oh, brother...after hearing about this ridicul...",0,"[101, 2821, 1010, 2567, 1012, 1012, 1012, 2044...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
3995,A hit at the time but now better categorised a...,1,"[101, 1037, 2718, 2012, 1996, 2051, 2021, 2085...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3996,I love this movie like no other. Another time ...,1,"[101, 1045, 2293, 2023, 3185, 2066, 2053, 2060...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3997,This film and it's sequel Barry Mckenzie holds...,1,"[101, 2023, 2143, 1998, 2009, 1005, 1055, 8297...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3998,'The Adventures Of Barry McKenzie' started lif...,1,"[101, 1005, 1996, 7357, 1997, 6287, 18506, 100...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Preparing training settings with TrainingArguments and Trainer class

TrainingArguments is the subset of the arguments we use in our example scripts which relate to the training loop itself.

In [16]:
from transformers import TrainingArguments, Trainer
from torch import cuda

training_args = TrainingArguments(
    # The output directory where the model predictions and checkpoints will be written
    output_dir="./MyIMDBModel",
    do_train=True,
    do_eval=True,
    #  The number of epochs, defaults to 3.0
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    # Number of steps used for a linear warmup
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy="steps",
    # TensorBoard log directory
    logging_dir="./logs",
    logging_steps=50,
    # other options : no, steps
    evaluation_strategy="steps",
    save_strategy="epoch",
    fp16=cuda.is_available(),
    report_to=["tensorboard"],
)
training_args

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=50,
eval_strategy=IntervalStrategy.STEPS,
eval_use

Let's design our evaluation metrics as follows:

In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="macro"
    )
    acc = accuracy_score(labels, preds)
    return {"Accuracy": acc, "F1": f1, "Precision": precision, "Recall": recall}

In [18]:
trainer = Trainer(
    # the pre-trained model that will be fine-tuned
    model=model,
    # training arguments that we defined above
    args=training_args,
    # training and validation dataset
    train_dataset=enc_train,
    eval_dataset=enc_val,
    compute_metrics=compute_metrics,
)
trainer

In [19]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

results = trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.672300,0.592210,0.692000,0.678626,0.730344,0.692000
100,0.423500,0.304323,0.868000,0.867809,0.870138,0.868000
150,0.278800,0.317423,0.886000,0.885817,0.888486,0.886000
200,0.191700,0.276727,0.894000,0.893939,0.894910,0.894000
250,0.208900,0.323915,0.875000,0.874563,0.880295,0.875000
300,0.081900,0.359451,0.897000,0.896925,0.898161,0.897000
350,0.091500,0.368741,0.895000,0.894997,0.895040,0.895000


In [20]:
!tree -a ./

./
├── logs
│   └── events.out.tfevents.1740651805.abdd3ccc3a1e.444.0
└── MyIMDBModel
    ├── checkpoint-125
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    ├── checkpoint-250
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    └── checkpoint-375
        ├── config.json
        ├── model.safetensors
        ├── optimizer.pt
        ├── rng_state.pth
        ├── scheduler.pt
        ├── trainer_state.json
        └── training_args.bin

5 directories, 22 files


In [21]:
q = [trainer.evaluate(eval_dataset=data) for data in [enc_train, enc_val, enc_test]]
pd.DataFrame(q, index=["train", "val", "test"]).iloc[:, :5]

,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.044256,0.99025,0.990250,0.990271,0.99025
val,0.367496,0.89400,0.893998,0.894025,0.89400
test,0.299468,0.90700,0.906998,0.907041,0.90700


In [22]:
# saving the best fine-tuned model & tokenizer
model_save_path = "MyBestIMDBModel"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

('MyBestIMDBModel/tokenizer_config.json',
 'MyBestIMDBModel/special_tokens_map.json',
 'MyBestIMDBModel/vocab.txt',
 'MyBestIMDBModel/added_tokens.json',
 'MyBestIMDBModel/tokenizer.json')

In [23]:
!tree -a ./

./
├── logs
│   ├── events.out.tfevents.1740651805.abdd3ccc3a1e.444.0
│   └── events.out.tfevents.1740652030.abdd3ccc3a1e.444.1
├── MyBestIMDBModel
│   ├── config.json
│   ├── model.safetensors
│   ├── special_tokens_map.json
│   ├── tokenizer_config.json
│   ├── tokenizer.json
│   ├── training_args.bin
│   └── vocab.txt
└── MyIMDBModel
    ├── checkpoint-125
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    ├── checkpoint-250
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    └── checkpoint-375
        ├── config.json
        ├── model.safetensors
        ├── optimizer.pt
        ├── rng_state.pth
        ├── scheduler.pt
        ├── trainer_state.json
        └── training_args.bin

6 directories, 30 

In [24]:
# saving the best fine-tuned model & tokenizer
model_save_path = "MyBestIMDBModel"
trainer.save_model(model_save_path+"model")
tokenizer.save_pretrained(model_save_path+"tokenizer")

('MyBestIMDBModeltokenizer/tokenizer_config.json',
 'MyBestIMDBModeltokenizer/special_tokens_map.json',
 'MyBestIMDBModeltokenizer/vocab.txt',
 'MyBestIMDBModeltokenizer/added_tokens.json',
 'MyBestIMDBModeltokenizer/tokenizer.json')

In [25]:
!tree -a ./

./
├── logs
│   ├── events.out.tfevents.1740651805.abdd3ccc3a1e.444.0
│   └── events.out.tfevents.1740652030.abdd3ccc3a1e.444.1
├── MyBestIMDBModel
│   ├── config.json
│   ├── model.safetensors
│   ├── special_tokens_map.json
│   ├── tokenizer_config.json
│   ├── tokenizer.json
│   ├── training_args.bin
│   └── vocab.txt
├── MyBestIMDBModelmodel
│   ├── config.json
│   ├── model.safetensors
│   └── training_args.bin
├── MyBestIMDBModeltokenizer
│   ├── special_tokens_map.json
│   ├── tokenizer_config.json
│   ├── tokenizer.json
│   └── vocab.txt
└── MyIMDBModel
    ├── checkpoint-125
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    ├── checkpoint-250
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    

## 验证

In [28]:
from torch import cuda

device = "cuda" if cuda.is_available() else "cpu"
print(device)

def get_prediction(text):
    inputs = tokenizer(
        text, padding=True, truncation=True, max_length=250, return_tensors="pt"
    ).to(device)
    outputs = model(inputs["input_ids"].to(device), inputs["attention_mask"].to(device))
    probs = outputs[0].softmax(1)
    return probs, probs.argmax()

cuda


In [30]:
model.to(device)
text = "I didn't like the movie since it bored me "
get_prediction(text)

(tensor([[0.9562, 0.0438]], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(0, device='cuda:0'))

In [31]:
get_prediction(text)[1].item()

0

## Use the model with pipeline

In [32]:
!tree -a ./

./
├── logs
│   ├── events.out.tfevents.1740651805.abdd3ccc3a1e.444.0
│   └── events.out.tfevents.1740652030.abdd3ccc3a1e.444.1
├── MyBestIMDBModel
│   ├── config.json
│   ├── model.safetensors
│   ├── special_tokens_map.json
│   ├── tokenizer_config.json
│   ├── tokenizer.json
│   ├── training_args.bin
│   └── vocab.txt
├── MyBestIMDBModelmodel
│   ├── config.json
│   ├── model.safetensors
│   └── training_args.bin
├── MyBestIMDBModeltokenizer
│   ├── special_tokens_map.json
│   ├── tokenizer_config.json
│   ├── tokenizer.json
│   └── vocab.txt
└── MyIMDBModel
    ├── checkpoint-125
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    ├── checkpoint-250
    │   ├── config.json
    │   ├── model.safetensors
    │   ├── optimizer.pt
    │   ├── rng_state.pth
    │   ├── scheduler.pt
    │   ├── trainer_state.json
    │   └── training_args.bin
    

In [33]:
from transformers import (
    pipeline,
    DistilBertForSequenceClassification,
    DistilBertTokenizerFast,
)

model = DistilBertForSequenceClassification.from_pretrained("MyBestIMDBModel")
tokenizer = DistilBertTokenizerFast.from_pretrained("MyBestIMDBModel")
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
nlp("the movie was very impressive")

Device set to use cuda:0


[{'label': 'POS', 'score': 0.9680010676383972}]

In [34]:
nlp("the script of the picture was very poor")

[{'label': 'NEG', 'score': 0.9873886108398438}]